03 言葉の最小単位：単語からnグラム、そしてフレーズへ
============================================

* ここでは、nグラムに対する特徴選択のフィルタ法として`コロケーション抽出`について説明する

    * ここまでは`トークン`という言葉を曖昧なまま使ってきたが、nグラムについての理解を正確にするために、`トークン`について正式に説明する
    
    * `トークン`：生データから単語を抽出するまでのプロセスにおいて、中間生成物として出てくる概念

## 1. パース処理とトークン化

* `テキストデータ`は、コンピュータ上では単なる文字列であり、ここから単語やnグラムを抽出するまでには長い道のありがある

    * 例)JSON、HTMLのような半構造データとして与えられることもある
    
    * 例)電子メールにはFrom、To、Subjectなどのヘッダフィールドが含まれている
    
* `パース`：テキストデータから本当に解析対象としたい文章のみを文字列として抽出する

    * この処理を行うことで、テキストデータから不要な構造が取り除かれ、文章だけを含むようになる

* `パース処理`によって抽出された文章は、`トークン`の配列に変換される

    * この処理のことを、`トークン化`と呼ぶ
    
* `トークン`とは、文章の中でひとまとまりの意味を持つ文字列のこと(通常は単語)

    * 文章中の単語を見分けるのは人間にとって簡単だが、コンピュータにとっては文章は単なる文字列オブジェクト
    
    * 従って、文章中から単語を抽出するルールを教える必要がある
    
* 通常、このルールとして、文章中で単語の境界となる`区切り文字`を定めるだけで十分

    * 区切り文字がわかれば、プログラムは文章を単語に分解できる
    
    * 普通の英語の場合、単語と単語を分ける境界は空白文字と句読点(これらを区切り文字として指定する)
    
    * 区切り文字に何を含めるかによって、トークン化の結果が変わる

* トークンの配列は、単語やnグラムに変換される

    * トークンをそのまま単語と考えることができれば、ステミングなどの処理を挟むこともできる
    
    * nグラムは連続したn個のトークンの配列
    
    * nグラムの抽出が目的である場合、トークン化の処理を文書ごとでなく文章ごとに行う必要がある
    
        * これは、文章の境界を超えてnグラムを作りたくないため
        
        * また、word2vecのような高度な特徴量生成法は、文章ごとだけでなく、段落ごとに適用されるものもある

## 2. フレーズ検出のためのコロケーション抽出

* nグラムから生成される特徴量の数を抑えるために、`フレーズ検出`が役に立つ

* `フレーズ`：単語の集まりのことであり、nグラムのうち有用なフレーズだけを残すことで特徴量の数を削減できる

    * 有用なフレーズのことを、`コロケーション`と呼ぶ
    
    > 物事について言及するための寛容的な方法に対応する2つ以上の単語からなる表現

* コロケーションとは、単なる単語の組み合わせ以上の意味を持つフレーズ

    * 例)"strong tea"(濃いお茶)は、"strong"(力強い)と"tea"(お茶)を組み合わせた意味とは異なる(ゆえに、"strong tea"はコロケーション)

* コロケーションから生成された特徴量は有用である可能性が高くなる

    * 1つ目は、コロケーションのリストを手作業で作成する
    
        * コーパスがドメインに特化しており、難解な用語を含んでいる場合に有効
        
        * ただし、頻繁に更新されるものには向いていない
        
    * この20年間で、コロケーション抽出のための統計的手法が開発された
         
         * これにより、コロケーションのリストを手作業で用意しなくても、頻繁に更新されるデータに対してコロケーション抽出ができる

### 2.1. 頻度に基づく方法

* 単純なコロケーション抽出として、頻度を用いる方法が考えられる

    * これは、テキストに頻繁に現れるnグラムは有用なフレーズであるという考え方
    
    * しかし、実際には頻繁に現れるものは必ずしも有用とは言えない
    
* 以下の図に、Yelpデータセットにおける頻度の高いバイグラムを示す

    * 上位10個のバイグラムは、フレーズとしてほとんど意味を持たない

In [1]:
# pandasで上位10個までのものを表示する

### 2.2.仮説検定を用いたコロケーション抽出

* 頻度によるコロケーション抽出はうまくいかない

    * 意味のあるフレーズを抽出するためには統計的な手法を使う必要がある
    


* `仮説検定`：データの統計量に基づいて、質問に「はい」か「いいえ」で答える手法

    * 2つの単語が連続して現れるのが偶然よりも多いかどうかを判定する
    
    * 観測されたデータは確率分布から抽出されたという仮定を置く
    
    * 例)仮説検定の結果は、「2つのデータセットが同じ分布から抽出された確率は95%である」のようになる

* 仮説検定を用いたコロケーション抽出には多くの手法が提案されている

* 現在最も利用されているのは、尤度比検定に基づく手法

    * この手法では、与えられた単語のペアに対して、観測データ上で次の仮説を検定する
    
    

* 仮説1(帰無仮説)：単語1は単語2とは独立して現れる

* 仮説2(対立仮説)：単語1が現れると、単語2が出現する確率は変わる

* この仮説を、尤度比検定を用いて検定する

    * これにより、次の質問に対する答えが得られる
    
    > 与えられたコーパスにおける観測された単語の出現回数を生成するモデルとして、2つの単語を独立と仮定するモデルと独立でないと仮定するモデルでは、どちらが確からしいか？

* 文章では正確なところが伝わりにくいので、数学的に記述する

* 単語1を$w_1$、単語2を$w_2$とした時、それぞれの仮説は以下の式で表される

  

* 帰無仮説(独立)：$H_{null}:P(w_2| w_1) = P(w_2 | not\ w_1)$

* 対立仮説(独立でない)：$H_{alternate}:P(w_2 | w_1)\neq P(w_2 | not\ w_1)$

* この仮説を検定するために、尤度比検定では次の統計量が使われる

\begin{eqnarray}
\log \lambda = \log \frac{L(Data; H_{null})}{L(Data; H_{alternate})}
\end{eqnarray}

* 尤度関数$L(Data; H)$は、仮説$H$のもとで、データセット$Data$が得られる確率を表す

    * 実際にこの確率を計算するには、データ生成モデルについて仮定を置く必要がある
    
* 最も簡単なデータ生成モデルは、`二項モデル`

    * これは、各単語に対してコインを投げ、コインが表ならばその単語を挿入し、裏なら別の単語を挿入することでデータが生成されるモデル
    
    * このモデルでは、単語の出現回数は`二項分布`に従う
    
    * この二項分布のパラメータは、単語の総数、関心のある単語の出現回数、および表が出る確率によって完全に決定される

尤度比検定を用いたコロケーション抽出のアルゴリズムを以下に示す

1. 全ての単語$w$に対して`生起確率`$P(w)$を求める

1. 全てのバイグラム$(w_1, w_2)$に対して条件付き確率$P(w_2 | w_1)$を求める

1. 全てのバイグラムに対して対数尤度比$\log \lambda$を求める

1. 対数尤度比に基づいてソートする

1. 対数尤度比の小さいバイグラムをコロケーションとする

> 尤度比検定を理解する
>
> 尤度比検定で比較されるのは確率パラメータそのものではなく、そのパラメータ(および仮定されたデータ生成モデル)の下で観測データが生成される確率。

* 他にも、自己相互情報量に基づく統計的アプローチがあるが、現実世界のテキストコーパスに常に存在するレアな単語に非常に敏感

* 統計的なコロケーション抽出法では、あらかじめ候補となるフレーズのリストを用意しておく必要がある

    * このリストとしてnグラムを使用するのが最も簡単
    
    * ただし、候補をnグラムに限定すると、"knock door"のような連続しないコロケーションを抽出できない
    
* 実用上、nグラムとして使われるのはバイグラムかトライグラムまであり、それ以上を使う人は滅多にいない

    * 長い候補を生成するためには、チャンク化などの他の方法が用いられる

### 2.3.チャンク化と品詞タグ付け

* コロケーション抽出の候補フレーズを作成するために`チャンク化`を行うこともある

    * これは、品詞に基づいてトークンの配列を抽出する手法

* 例)カテゴリ分類のタスクでは、冠詞や前置詞から特徴量を生成しても精度に貢献しないと考えれられる

    * フレーズに対しても同様のことが言えるため、コロケーション抽出の候補として「名詞句」だけを抽出する
    
    * 「名詞句」を見つけるために、トークンに品詞タグを付与し、名詞トークンをその近くにある単語とグループ化する
    
    * このグループを`チャンク(塊)`と呼び、チャンクを作成することを`チャンク化`と呼ぶ
    
* Pythonでは、`NLTK`、`spaCy`、`TextBlob`などのライブラリを使って品詞タグ付けとチャンク化を簡単に行うことができる

    * 例)Yelpデータセットを使って、品詞タグを使ったチャンク化を行う(spaCy、TextBlob)

In [ ]:
import pandas as pd
import json

# 最初の10レビューを読み込む
with open('data/yelp/yelp_academic_dataset_review.json') as f:
    js = []
    for i in range(10):
        js.append(json.loads(f.readline()))
review_df = pd.DataFrame(js)

# まずは Spacy を使った方法
import spacy
# 言語モデル（英語）を読み込む
nlp = spacy.load('en')

# spaCy の言語モデルを使ってテキストから Pandas Series を作成する
doc_df = review_df['text'].apply(nlp)

# spaCy は細かい品詞タグを .pos_ で、粗い品詞タグを .tag_ で提供します
for doc in doc_df[4]:
    print([doc.text, doc.pos_, doc.tag_])

# spaCy は基本的な名詞句も .noun_chunks で提供します
print([chunk for chunk in doc_df[4].noun_chunks])

In [ ]:
# TextBlob ライブラリを使って同じことができる
from textblob import TextBlob

# TextBlob はデフォルトでは PatternTagger を使ってタグ付けを行う。
# これは今回の例ではうまくいくが、文法の正しくない文章を含む場合は 
# NLTKTagger を使うことをおすすめする。
blob_df = review_df['text'].apply(TextBlob)

blob_df[4].tags

In [ ]:
print([np for np in blob_df[4].noun_phrases])

* 抽出された名詞句は、ライブラリごとに少し異なることがわかる

    * spaCyは"a"や"the"のような英語の共通単語を名詞句に含むが、TextBlobは含まない
    
    * これは、各ライブラリにおいて、何を名詞句とみなすかというルールエンジンが異なるため
    
* 品詞の関係を独自に定義してチャンクを抽出することもできる

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/04/21 |